## Data Modelling & Analysis

#### I scraped the data from Cricinfo, wikipedia as well as other sites.
### steps followed are as following:

1. for each year (2018 - 2023) I scraped all the matched played in that year across all teams,
check the function scrape_match_dataset. 
(which also contained links to the team profile as well as links for the Match detailed scorecard)

2. for each year I went to each match link and scraped both the batting scorecard and bowling scorecard.
(Data scraped in such a way that minimum effort is required in data modelling)
(check the fucntions - scrape_batting_summary, srape_bowling_summary)

3. After this from Match scorecard i also scraped the player links which helped me scrape cruiucla details about each player such as their stats, 
their nationality their preference order etc.

4. Scraped National falgs for each team

5. Because of issue of Lazy loading of player's images had to scrape the pictures using Selenium where I intialized delay time as 20 secounds and it will keep on increasing the time by 5 seconds till 60 seconds, if image is still not loading. 
(code in second notebook along with data modelling and analysis)


### Data Modelling

1. As the final goal of the projct is to create beautiful visual in Tableau, and as we are working with more than 12 tables it's important that data is modelled properly. For that purpose instead of creating ine big massive dataset I scraped data keeping this in mind beforehand and thus reduced the effort spent.

2. The main objective of the anlalysis was to rank the players based on their Past 5 years performance and utilizing that data to built best playing 11 for each team for different playing conditions like (This Analysis done seperatley on excel sheet):

- Balanced - Balnced squad of bolwers, batsmen and Allrounders (3+3+5)
- Batting Max - Focus a little more on bowling (3 bolwers + 2 allrounders + 6 batsmen)
- Bowling Max - focus mor on strengthening the bowling (4 bolwers, 2 allrounders, 5 batsmen)
- Batting Super - Focus on strnegthening Batting at highest level (2 bowlers + 3 allrounders + 6 batsmen)
- My Best 11 - My best 11 created based on the data, my intution as well as my knowledge on cricket.



In [289]:
import pandas as pd
from fuzzywuzzy import process

### Loading The Dataset

In [2]:
df_players = pd.read_csv("player_details_data.csv")
df_batting = pd.read_csv("batting_scorecard_data.csv")
df_bowling = pd.read_csv("bowling_scorecard_data.csv")
df_fielding = pd.read_csv("fielding_stats_data.csv")

### Cleaning the Dataset& Aggregating

In [3]:
df_players.columns

Index(['Unnamed: 0', 'Name', 'Born', 'Age', 'Batting_Style', 'Bowling_Style',
       'Playing_Role', 'National_Team', 'player_img', 'career_start',
       'career_latest'],
      dtype='object')

#### Aggregating the Dataaset (Df_batting)

In [4]:
df_players = df_players.drop(columns=["Unnamed: 0"])

In [5]:
df_batting['R'] = df_batting['R'].str.replace('-', '0')
df_batting['B'] = df_batting['B'].str.replace('-', '0')
df_batting['4s'] = df_batting['4s'].str.replace('-', '0')
df_batting['6s'] = df_batting['6s'].str.replace('-', '0')
df_batting['SR'] = df_batting['SR'].str.replace('-', '0')

In [6]:
df_batting['R'] = df_batting['R'].astype(int)
df_batting['B'] = df_batting['B'].astype(int)
df_batting['4s'] = df_batting['4s'].astype(int)
df_batting['6s'] = df_batting['6s'].astype(int)
df_batting['SR'] = df_batting['SR'].astype(float)

In [7]:
# Create a new column for number of 100's and nu,mber of 50's
def map_100(value):
    if value >= 100:
        return 1
    else:
        return 0
    
def map_50(value):
    if value >= 50 and value < 100:
        return 1
    else:
        return 0    
    
# Apply the custom function to create the new column
df_batting["100s"] = df_batting['R'].apply(map_100)
df_batting['50s'] = df_batting['R'].apply(map_50)

In [8]:
df_batting

,BATTING,R,B,4s,6s,SR,team,Match_id,odi_year,100s,50s
0,Martin Guptill,48,72,4,2,66.66,New Zealand,ODI # 3946,2018-2018,0,0
1,Colin Munro,58,35,6,2,165.71,New Zealand,ODI # 3946,2018-2018,0,1
2,Kane Williamson,115,117,8,1,98.29,New Zealand,ODI # 3946,2018-2018,1,0
3,Ross Taylor,12,14,0,0,85.71,New Zealand,ODI # 3946,2018-2018,0,0
4,Tom Latham,3,7,0,0,42.85,New Zealand,ODI # 3946,2018-2018,0,0
...,...,...,...,...,...,...,...,...,...,...,...
12037,Mohammad Naim,0,8,0,0,0.00,Bangladesh,ODI # 4621,2023-2023,0,0
12038,Litton Das,53,60,3,2,88.33,Bangladesh,ODI # 4621,2023-2023,0,1
12039,Najmul Hossain Shanto,11,15,2,0,73.33,Bangladesh,ODI # 4621,2023-2023,0,0
12040,Shakib Al Hasan,39,39,5,0,100.00,Bangladesh,ODI # 4621,2023-2023,0,0


In [9]:
df_batting_agg = df_batting.groupby(['BATTING',"team"], as_index=False).agg({
    'R': 'sum',
    'B': 'sum',
    '4s': 'sum',
    '6s': 'sum',
    'SR': 'mean',
    "100s":'sum',
    "50s":"sum",
    'Match_id': 'nunique' 
})

In [10]:
df_batting_agg

,BATTING,team,R,B,4s,6s,SR,100s,50s,Match_id
0,AB de Villiers,South Africa,62,59,5,2,106.126667,0,0,3
1,Aamer Yamin,Pakistan,32,27,3,1,118.510000,0,0,1
2,Aamir Kaleem,Oman,18,24,2,0,71.875000,0,0,2
3,Aarif Sheikh,Nepal,878,1406,53,4,56.918250,0,6,40
4,Aaron Finch,Australia,2481,2896,215,66,68.909344,9,13,61
...,...,...,...,...,...,...,...,...,...,...
820,Zawar Farid,United Arab Emirates,132,139,15,4,96.390000,0,0,8
821,Zeeshan Maqsood,Oman,1127,1611,90,22,64.232683,2,6,41
822,Zhivago Groenewald,Namibia,4,19,0,0,58.330000,0,0,2
823,Zia-ur-Rehman,Afghanistan,5,30,1,0,16.660000,0,0,1


In [11]:
sum(df_batting_agg[["BATTING","team"]].duplicated())

0

#### Aggregating the Dataaset (Df_bowling)

In [12]:
df_bowling.columns

Index(['BOWLING', 'O', 'M', 'R', 'W', 'ECON', '0s', '4s', '6s', 'WD', 'NB',
       'Match_id', 'team', 'odi_year'],
      dtype='object')

In [13]:
df_bowling.dtypes

BOWLING      object
O           float64
M             int64
R             int64
W             int64
ECON        float64
0s           object
4s           object
6s           object
WD            int64
NB            int64
Match_id     object
team         object
odi_year     object
dtype: object

In [14]:
df_bowling['0s'] = df_bowling['0s'].str.replace('-', '0')
df_bowling['4s'] = df_bowling['4s'].str.replace('-', '0')
df_bowling['6s'] = df_bowling['6s'].str.replace('-', '0')

df_bowling['0s'] = df_bowling['0s'].astype(int)
df_bowling['4s'] = df_bowling['4s'].astype(int)
df_bowling['6s'] = df_bowling['6s'].astype(int)

In [15]:
## New column for number of 5 wickets
def map_5w(value):
    if value >= 5:
        return 1
    else:
        return 0
    
# Apply the custom function to create the new column
df_bowling["5w_haul"] = df_bowling['W'].apply(map_5w)

In [16]:
10.5//1

10.0

In [17]:
df_bowling['O']%1

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
8067    0.0
8068    0.3
8069    0.0
8070    0.0
8071    0.0
Name: O, Length: 8072, dtype: float64

In [18]:
## New column for number of 5 wickets
# Apply the custom function to create the new column
df_bowling["balls_bowled"] = ((df_bowling['O']//1)*6 + (df_bowling['O']%1)*10)

In [19]:
df_bowling.columns

Index(['BOWLING', 'O', 'M', 'R', 'W', 'ECON', '0s', '4s', '6s', 'WD', 'NB',
       'Match_id', 'team', 'odi_year', '5w_haul', 'balls_bowled'],
      dtype='object')

In [20]:
df_bowling_agg = df_bowling.groupby(['BOWLING',"team"], as_index=False).agg({
    'O': 'sum',
    'M': 'sum',
    'R': 'sum',
    'W': 'sum',
    'ECON': 'mean',
    '0s': 'sum',
    '4s': 'sum',
    '6s': 'sum',
    'WD': 'sum',
    'NB': 'sum',
    "5w_haul":"sum",
    "balls_bowled":"sum",
    "Match_id":'nunique'
})

In [21]:
df_bowling_agg[df_bowling_agg["5w_haul"]>1][["BOWLING","team","5w_haul"]].sort_values(by="5w_haul",ascending = False)

,BOWLING,team,5w_haul
367,Mitchell Starc,Australia,4
85,Bilal Khan,Oman,4
488,Sandeep Lamichhane,Nepal,3
590,Wanindu Hasaranga,Sri Lanka,3
282,Karan KC,Nepal,3
509,Shakib Al Hasan,Bangladesh,3
32,Ali Khan,United States of America,2
573,Trent Boult,New Zealand,2
505,Shaheen Shah Afridi,Pakistan,2
500,Semo Kamea,Papua New Guinea,2


#### Dropping duplicates in df_players

In [22]:
df_players

,Name,Born,Age,Batting_Style,Bowling_Style,Playing_Role,National_Team,player_img,career_start,career_latest
0,Martin Guptill,"September 30, 1986, Auckland",36y 308d,Right hand Bat,Right arm Offbreak,Opening Batter,New Zealand,https://upload.wikimedia.org/wikipedia/commons...,2009,2022
1,Colin Munro,"March 11, 1987, Durban",36y 146d,Left hand Bat,Right arm Medium fast,Opening Batter,New Zealand,https://upload.wikimedia.org/wikipedia/commons...,2012,2020
2,Kane Williamson,"August 08, 1990, Tauranga",32y 361d,Right hand Bat,Right arm Offbreak,Top order Batter,New Zealand,https://upload.wikimedia.org/wikipedia/commons...,2010,2023
3,Ross Taylor,"March 08, 1984, Lower Hutt, Wellington",39y 149d,Right hand Bat,Right arm Offbreak,Middle order Batter,New Zealand,https://upload.wikimedia.org/wikipedia/commons...,2006,2022
4,Tom Latham,"April 02, 1992, Christchurch, Canterbury",31y 124d,Left hand Bat,Right arm Medium,Wicketkeeper Batter,New Zealand,NaN,2012,2023
...,...,...,...,...,...,...,...,...,...,...
2224,Tim Southee,"December 11, 1988, Whangarei, Northland",34y 236d,Right hand Bat,Right arm Medium fast,Bowler,New Zealand,https://upload.wikimedia.org/wikipedia/commons...,2008,2023
2225,Todd Astle,"September 24, 1986, Palmerston North, New Zealand",36y 314d,Right hand Bat,Legbreak,Bowler,New Zealand,NaN,2012,2021
2226,Azhar Ali,"February 19, 1985, Lahore, Punjab",38y 166d,Right hand Bat,Legbreak,Top order Batter,Pakistan,https://upload.wikimedia.org/wikipedia/commons...,2010,2022
2227,Fakhar Zaman,"April 10, 1990, Mardan, North-West Frontier Pr...",33y 116d,Left hand Bat,Slow Left arm Orthodox,Opening Batter,Pakistan,NaN,2017,2023


In [23]:
# There is No duplicates in this table !!!

In [24]:
duplicate = df_players.duplicated()
print(df_players[duplicate])

                 Name                                               Born  \
362    Martin Guptill                       September 30, 1986, Auckland   
363       Colin Munro                             March 11, 1987, Durban   
364   Kane Williamson                          August 08, 1990, Tauranga   
365       Ross Taylor             March 08, 1984, Lower Hutt, Wellington   
366    Henry Nicholls                    November 15, 1991, Christchurch   
...               ...                                                ...   
2224      Tim Southee            December 11, 1988, Whangarei, Northland   
2225       Todd Astle  September 24, 1986, Palmerston North, New Zealand   
2226        Azhar Ali                  February 19, 1985, Lahore, Punjab   
2227     Fakhar Zaman  April 10, 1990, Mardan, North-West Frontier Pr...   
2228       Babar Azam                   October 15, 1994, Lahore, Punjab   

           Age   Batting_Style           Bowling_Style         Playing_Role  \
362   36

In [25]:
df_players = df_players.drop_duplicates()

In [26]:
sum((df_batting.duplicated()))

0

In [27]:
sum((df_bowling.duplicated()))

0

In [28]:
sum((df_players.duplicated()))

0

In [29]:
# Part-1 Merge the 2 datasets primary key being the Name

In [30]:
df_players.columns

Index(['Name', 'Born', 'Age', 'Batting_Style', 'Bowling_Style', 'Playing_Role',
       'National_Team', 'player_img', 'career_start', 'career_latest'],
      dtype='object')

#### Filtering the dataset (Considering both years 2022 ands 2023 because many active players are incorrectly marked in year 2022 )

In [31]:
df_players = df_players[(df_players["career_latest"] == 2023) | (df_players["career_latest"] == 2022)]

In [32]:
df_players

,Name,Born,Age,Batting_Style,Bowling_Style,Playing_Role,National_Team,player_img,career_start,career_latest
0,Martin Guptill,"September 30, 1986, Auckland",36y 308d,Right hand Bat,Right arm Offbreak,Opening Batter,New Zealand,https://upload.wikimedia.org/wikipedia/commons...,2009,2022
2,Kane Williamson,"August 08, 1990, Tauranga",32y 361d,Right hand Bat,Right arm Offbreak,Top order Batter,New Zealand,https://upload.wikimedia.org/wikipedia/commons...,2010,2023
3,Ross Taylor,"March 08, 1984, Lower Hutt, Wellington",39y 149d,Right hand Bat,Right arm Offbreak,Middle order Batter,New Zealand,https://upload.wikimedia.org/wikipedia/commons...,2006,2022
4,Tom Latham,"April 02, 1992, Christchurch, Canterbury",31y 124d,Left hand Bat,Right arm Medium,Wicketkeeper Batter,New Zealand,NaN,2012,2023
5,Henry Nicholls,"November 15, 1991, Christchurch",31y 262d,Left hand Bat,Right arm Offbreak,Top order Batter,New Zealand,NaN,2015,2023
...,...,...,...,...,...,...,...,...,...,...
2211,Mohammad Saleem,"September 09, 2002, Baghlan",20y 329d,Right hand Bat,Right arm Fast,Bowler,Afghanistan,NaN,2022,2023
2212,Sahan Arachchige,"May 13, 1996, Ragama",27y 83d,Left hand Bat,Right arm Offbreak,Top order Batter,Sri Lanka,NaN,2023,2023
2214,Noah Croes,"December 13, 1999",23y 234d,Right hand Bat,NaN,Batter,Netherlands,NaN,2023,2023
2215,Abdul Rahman,"November 22, 2001",21y 255d,Right hand Bat,Right arm Fast medium,Bowler,Afghanistan,NaN,2023,2023


In [33]:
df_batting_agg.columns

Index(['BATTING', 'team', 'R', 'B', '4s', '6s', 'SR', '100s', '50s',
       'Match_id'],
      dtype='object')

In [34]:
df_bowling_agg.columns

Index(['BOWLING', 'team', 'O', 'M', 'R', 'W', 'ECON', '0s', '4s', '6s', 'WD',
       'NB', '5w_haul', 'balls_bowled', 'Match_id'],
      dtype='object')

In [35]:
# check
unique_values_df_batting = df_batting_agg['BATTING'].unique()

# Check if each unique value from dataset 1 exists in dataset 2
all_exist = all(value in df_players["Name"].values for value in unique_values_df_batting)

if all_exist:
    print("All unique values from dataset 1 exist in dataset 2.")
else:
    print("Not all unique values from dataset 1 exist in dataset 2.")

Not all unique values from dataset 1 exist in dataset 2.


In [36]:
# check
unique_values_df_bowling = df_bowling_agg['BOWLING'].unique()

# Check if each unique value from dataset 1 exists in dataset 2
all_exist = all(value in df_players["Name"].values for value in unique_values_df_bowling)

if all_exist:
    print("All unique values from dataset 1 exist in dataset 2.")
else:
    print("Not all unique values from dataset 1 exist in dataset 2.")

Not all unique values from dataset 1 exist in dataset 2.


In [37]:
# check
unique_values_df_fielding = df_fielding['Player_Name'].unique()

# Check if each unique value from dataset 1 exists in dataset 2
all_exist = all(value in df_players["Name"].values for value in unique_values_df_fielding)

if all_exist:
    print("All unique values from dataset 1 exist in dataset 2.")
else:
    print("Not all unique values from dataset 1 exist in dataset 2.")

Not all unique values from dataset 1 exist in dataset 2.


In [38]:
sum(df_players.duplicated())

0

In [39]:
df_players

,Name,Born,Age,Batting_Style,Bowling_Style,Playing_Role,National_Team,player_img,career_start,career_latest
0,Martin Guptill,"September 30, 1986, Auckland",36y 308d,Right hand Bat,Right arm Offbreak,Opening Batter,New Zealand,https://upload.wikimedia.org/wikipedia/commons...,2009,2022
2,Kane Williamson,"August 08, 1990, Tauranga",32y 361d,Right hand Bat,Right arm Offbreak,Top order Batter,New Zealand,https://upload.wikimedia.org/wikipedia/commons...,2010,2023
3,Ross Taylor,"March 08, 1984, Lower Hutt, Wellington",39y 149d,Right hand Bat,Right arm Offbreak,Middle order Batter,New Zealand,https://upload.wikimedia.org/wikipedia/commons...,2006,2022
4,Tom Latham,"April 02, 1992, Christchurch, Canterbury",31y 124d,Left hand Bat,Right arm Medium,Wicketkeeper Batter,New Zealand,NaN,2012,2023
5,Henry Nicholls,"November 15, 1991, Christchurch",31y 262d,Left hand Bat,Right arm Offbreak,Top order Batter,New Zealand,NaN,2015,2023
...,...,...,...,...,...,...,...,...,...,...
2211,Mohammad Saleem,"September 09, 2002, Baghlan",20y 329d,Right hand Bat,Right arm Fast,Bowler,Afghanistan,NaN,2022,2023
2212,Sahan Arachchige,"May 13, 1996, Ragama",27y 83d,Left hand Bat,Right arm Offbreak,Top order Batter,Sri Lanka,NaN,2023,2023
2214,Noah Croes,"December 13, 1999",23y 234d,Right hand Bat,NaN,Batter,Netherlands,NaN,2023,2023
2215,Abdul Rahman,"November 22, 2001",21y 255d,Right hand Bat,Right arm Fast medium,Bowler,Afghanistan,NaN,2023,2023


In [40]:
#so ready to merge these 3 datasets - primary key is Name of player

In [41]:
df_batting_agg.columns

Index(['BATTING', 'team', 'R', 'B', '4s', '6s', 'SR', '100s', '50s',
       'Match_id'],
      dtype='object')

In [42]:
sum(df_batting_agg.duplicated())

0

In [43]:
sum(df_bowling_agg.duplicated())

0

In [44]:
sum(df_fielding.duplicated())

0

In [45]:
df_fielding.columns

Index(['Player_Name', 'matches', 'catches', 'stumps', 'odi_year'], dtype='object')

In [46]:
df_fielding.dtypes

Player_Name    object
matches         int64
catches         int64
stumps          int64
odi_year       object
dtype: object

#### Aggregating the fielding dataset

In [47]:
df_fielding_agg = df_fielding.groupby(['Player_Name'], as_index=False).agg({
    'matches': 'mean',
    'catches': 'mean',
    'stumps': 'mean'
})

In [48]:
df_fielding_agg

,Player_Name,matches,catches,stumps
0,AB de Villiers,228.0,176.0,5.0
1,Aamer Yamin,4.0,0.0,0.0
2,Aamir Kaleem,3.0,2.0,0.0
3,Aarif Sheikh,47.0,21.0,0.0
4,Aaron Finch,146.0,71.0,0.0
...,...,...,...,...
839,Zawar Farid,10.0,1.0,0.0
840,Zeeshan Maqsood,42.0,22.0,0.0
841,Zhivago Groenewald,4.0,2.0,0.0
842,Zia-ur-Rehman,1.0,0.0,0.0


#### Joining the 4 datasets

In [49]:
# Perform the first join (A with B)
join_data = pd.merge(df_batting_agg, df_players, left_on=["BATTING", "team"], right_on=["Name", "National_Team"], how='right')
print(len(join_data))
# Reset index of join_data
join_data.reset_index(drop=True, inplace=True)

# Perform the second join (result of A-B join with C)
final_data2 = pd.merge(join_data, df_bowling_agg, left_on=["Name", "National_Team"], right_on=["BOWLING", "team"], how='left')
print(len(final_data2))
# Reset index of final_data2
final_data2.reset_index(drop=True, inplace=True)

# Perform the third join
final_data = pd.merge(final_data2, df_fielding_agg, left_on=["Name"], right_on=["Player_Name"], how='left')
print(len(final_data))
# Reset index of final_data
final_data.reset_index(drop=True, inplace=True)

631
631
631


In [50]:
final_data.columns

Index(['BATTING', 'team_x', 'R_x', 'B', '4s_x', '6s_x', 'SR', '100s', '50s',
       'Match_id_x', 'Name', 'Born', 'Age', 'Batting_Style', 'Bowling_Style',
       'Playing_Role', 'National_Team', 'player_img', 'career_start',
       'career_latest', 'BOWLING', 'team_y', 'O', 'M', 'R_y', 'W', 'ECON',
       '0s', '4s_y', '6s_y', 'WD', 'NB', '5w_haul', 'balls_bowled',
       'Match_id_y', 'Player_Name', 'matches', 'catches', 'stumps'],
      dtype='object')

In [51]:
#final_data[final_data["career_latest"] == 2023 or final_data["career_latest"] == 2022]

#### Considering players only part of the 10 participating worldcup teams

In [52]:
worldcup_teams = ["India", "Australia","South Africa","Sri Lanka","Netherlands","New Zealand","Pakistan","Bangladesh","Afghanistan","England"]

In [53]:
filtered_df = final_data[final_data['National_Team'].isin(worldcup_teams)]

In [54]:
filtered_df

,BATTING,team_x,R_x,B,4s_x,6s_x,SR,100s,50s,Match_id_x,...,6s_y,WD,NB,5w_haul,balls_bowled,Match_id_y,Player_Name,matches,catches,stumps
0,Martin Guptill,New Zealand,1793.0,2075.0,175.0,49.0,80.434082,6.0,7.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Martin Guptill,198.0,104.0,0.0
1,Kane Williamson,New Zealand,1838.0,2474.0,149.0,12.0,68.361951,4.0,10.0,41.0,...,0.0,1.0,0.0,0.0,198.0,6.0,Kane Williamson,161.0,64.0,0.0
2,Ross Taylor,New Zealand,1813.0,2048.0,147.0,26.0,78.242632,4.0,12.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Ross Taylor,236.0,142.0,0.0
3,Tom Latham,New Zealand,1858.0,2080.0,142.0,25.0,70.362807,3.0,12.0,57.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Tom Latham,130.0,111.0,15.0
4,Henry Nicholls,New Zealand,1323.0,1651.0,121.0,12.0,66.462083,1.0,10.0,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Henry Nicholls,66.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,8.0,1.0,0.0,62.0,2.0,Mohammad Saleem,2.0,1.0,0.0
627,Sahan Arachchige,Sri Lanka,57.0,71.0,4.0,0.0,80.280000,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,30.0,1.0,Sahan Arachchige,2.0,0.0,0.0
628,Noah Croes,Netherlands,7.0,6.0,1.0,0.0,116.660000,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Noah Croes,1.0,2.0,0.0
629,Abdul Rahman,Afghanistan,4.0,20.0,0.0,0.0,20.000000,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,24.0,1.0,Abdul Rahman,1.0,0.0,0.0


In [55]:
# merging the playing roles

In [56]:
filtered_df.columns

Index(['BATTING', 'team_x', 'R_x', 'B', '4s_x', '6s_x', 'SR', '100s', '50s',
       'Match_id_x', 'Name', 'Born', 'Age', 'Batting_Style', 'Bowling_Style',
       'Playing_Role', 'National_Team', 'player_img', 'career_start',
       'career_latest', 'BOWLING', 'team_y', 'O', 'M', 'R_y', 'W', 'ECON',
       '0s', '4s_y', '6s_y', 'WD', 'NB', '5w_haul', 'balls_bowled',
       'Match_id_y', 'Player_Name', 'matches', 'catches', 'stumps'],
      dtype='object')

#### Checking dataset for errrors

In [57]:
A = filtered_df["BATTING"].duplicated()
print(filtered_df[A])

    BATTING team_x  R_x   B  4s_x  6s_x  SR  100s  50s  Match_id_x  ... 6s_y  \
235     NaN    NaN  NaN NaN   NaN   NaN NaN   NaN  NaN         NaN  ...  2.0   
252     NaN    NaN  NaN NaN   NaN   NaN NaN   NaN  NaN         NaN  ...  0.0   
385     NaN    NaN  NaN NaN   NaN   NaN NaN   NaN  NaN         NaN  ...  1.0   
406     NaN    NaN  NaN NaN   NaN   NaN NaN   NaN  NaN         NaN  ...  0.0   
441     NaN    NaN  NaN NaN   NaN   NaN NaN   NaN  NaN         NaN  ...  0.0   
456     NaN    NaN  NaN NaN   NaN   NaN NaN   NaN  NaN         NaN  ...  1.0   
501     NaN    NaN  NaN NaN   NaN   NaN NaN   NaN  NaN         NaN  ...  0.0   
606     NaN    NaN  NaN NaN   NaN   NaN NaN   NaN  NaN         NaN  ...  0.0   
626     NaN    NaN  NaN NaN   NaN   NaN NaN   NaN  NaN         NaN  ...  1.0   

      WD   NB 5w_haul balls_bowled Match_id_y         Player_Name matches  \
235  3.0  0.0     0.0        114.0        2.0      Duanne Olivier     2.0   
252  5.0  0.0     0.0         60.0        1.0

In [58]:
filtered_df.columns

Index(['BATTING', 'team_x', 'R_x', 'B', '4s_x', '6s_x', 'SR', '100s', '50s',
       'Match_id_x', 'Name', 'Born', 'Age', 'Batting_Style', 'Bowling_Style',
       'Playing_Role', 'National_Team', 'player_img', 'career_start',
       'career_latest', 'BOWLING', 'team_y', 'O', 'M', 'R_y', 'W', 'ECON',
       '0s', '4s_y', '6s_y', 'WD', 'NB', '5w_haul', 'balls_bowled',
       'Match_id_y', 'Player_Name', 'matches', 'catches', 'stumps'],
      dtype='object')

In [59]:
filtered_df[filtered_df["team_x"] != filtered_df["team_y"]]

,BATTING,team_x,R_x,B,4s_x,6s_x,SR,100s,50s,Match_id_x,...,6s_y,WD,NB,5w_haul,balls_bowled,Match_id_y,Player_Name,matches,catches,stumps
0,Martin Guptill,New Zealand,1793.0,2075.0,175.0,49.0,80.434082,6.0,7.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Martin Guptill,198.0,104.0,0.0
2,Ross Taylor,New Zealand,1813.0,2048.0,147.0,26.0,78.242632,4.0,12.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Ross Taylor,236.0,142.0,0.0
3,Tom Latham,New Zealand,1858.0,2080.0,142.0,25.0,70.362807,3.0,12.0,57.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Tom Latham,130.0,111.0,15.0
4,Henry Nicholls,New Zealand,1323.0,1651.0,121.0,12.0,66.462083,1.0,10.0,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Henry Nicholls,66.0,24.0,0.0
7,Azhar Ali,Pakistan,12.0,30.0,2.0,0.0,62.500000,0.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Azhar Ali,53.0,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,Chad Bowes,New Zealand,84.0,97.0,12.0,0.0,70.640000,0.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Chad Bowes,4.0,4.0,0.0
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,48.0,1.0,Ihsanullah,8.5,5.0,0.0
609,Rony Talukdar,Bangladesh,4.0,14.0,1.0,0.0,28.570000,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Rony Talukdar,1.0,1.0,0.0
626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,8.0,1.0,0.0,62.0,2.0,Mohammad Saleem,2.0,1.0,0.0


In [60]:
filtered_df

,BATTING,team_x,R_x,B,4s_x,6s_x,SR,100s,50s,Match_id_x,...,6s_y,WD,NB,5w_haul,balls_bowled,Match_id_y,Player_Name,matches,catches,stumps
0,Martin Guptill,New Zealand,1793.0,2075.0,175.0,49.0,80.434082,6.0,7.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Martin Guptill,198.0,104.0,0.0
1,Kane Williamson,New Zealand,1838.0,2474.0,149.0,12.0,68.361951,4.0,10.0,41.0,...,0.0,1.0,0.0,0.0,198.0,6.0,Kane Williamson,161.0,64.0,0.0
2,Ross Taylor,New Zealand,1813.0,2048.0,147.0,26.0,78.242632,4.0,12.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Ross Taylor,236.0,142.0,0.0
3,Tom Latham,New Zealand,1858.0,2080.0,142.0,25.0,70.362807,3.0,12.0,57.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Tom Latham,130.0,111.0,15.0
4,Henry Nicholls,New Zealand,1323.0,1651.0,121.0,12.0,66.462083,1.0,10.0,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Henry Nicholls,66.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,8.0,1.0,0.0,62.0,2.0,Mohammad Saleem,2.0,1.0,0.0
627,Sahan Arachchige,Sri Lanka,57.0,71.0,4.0,0.0,80.280000,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,30.0,1.0,Sahan Arachchige,2.0,0.0,0.0
628,Noah Croes,Netherlands,7.0,6.0,1.0,0.0,116.660000,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Noah Croes,1.0,2.0,0.0
629,Abdul Rahman,Afghanistan,4.0,20.0,0.0,0.0,20.000000,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,24.0,1.0,Abdul Rahman,1.0,0.0,0.0


In [61]:
# Group by 'Team' and count the number of rows for each team
team_counts = filtered_df.groupby('National_Team').size().reset_index(name='Count')
team_counts

,National_Team,Count
0,Afghanistan,28
1,Australia,29
2,Bangladesh,28
3,England,37
4,India,38
5,Netherlands,28
6,New Zealand,34
7,Pakistan,32
8,South Africa,31
9,Sri Lanka,42


In [62]:
filtered_df[filtered_df["team_x"] != filtered_df["team_y"]]

,BATTING,team_x,R_x,B,4s_x,6s_x,SR,100s,50s,Match_id_x,...,6s_y,WD,NB,5w_haul,balls_bowled,Match_id_y,Player_Name,matches,catches,stumps
0,Martin Guptill,New Zealand,1793.0,2075.0,175.0,49.0,80.434082,6.0,7.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Martin Guptill,198.0,104.0,0.0
2,Ross Taylor,New Zealand,1813.0,2048.0,147.0,26.0,78.242632,4.0,12.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Ross Taylor,236.0,142.0,0.0
3,Tom Latham,New Zealand,1858.0,2080.0,142.0,25.0,70.362807,3.0,12.0,57.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Tom Latham,130.0,111.0,15.0
4,Henry Nicholls,New Zealand,1323.0,1651.0,121.0,12.0,66.462083,1.0,10.0,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Henry Nicholls,66.0,24.0,0.0
7,Azhar Ali,Pakistan,12.0,30.0,2.0,0.0,62.500000,0.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Azhar Ali,53.0,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,Chad Bowes,New Zealand,84.0,97.0,12.0,0.0,70.640000,0.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Chad Bowes,4.0,4.0,0.0
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,48.0,1.0,Ihsanullah,8.5,5.0,0.0
609,Rony Talukdar,Bangladesh,4.0,14.0,1.0,0.0,28.570000,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Rony Talukdar,1.0,1.0,0.0
626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,8.0,1.0,0.0,62.0,2.0,Mohammad Saleem,2.0,1.0,0.0


In [63]:
filtered_df[filtered_df["team_x"] != filtered_df["National_Team"]]

,BATTING,team_x,R_x,B,4s_x,6s_x,SR,100s,50s,Match_id_x,...,6s_y,WD,NB,5w_haul,balls_bowled,Match_id_y,Player_Name,matches,catches,stumps
184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Michael Rippon,9.0,4.0,0.0
235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,3.0,0.0,0.0,114.0,2.0,Duanne Olivier,2.0,0.0,0.0
252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,5.0,0.0,0.0,60.0,1.0,Zahir Khan,1.0,0.0,0.0
385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,54.0,1.0,Usman Qadir,1.0,1.0,0.0
406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,5.0,1.0,0.0,48.0,1.0,Lizaad Williams,1.0,0.0,0.0
441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,6.0,1.0,Chamika Gunasekara,1.0,0.0,0.0
456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,46.0,1.0,Qais Ahmad,1.0,0.0,0.0
501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,3.0,0.0,0.0,54.0,1.0,David Payne,1.0,0.0,0.0
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,48.0,1.0,Ihsanullah,8.5,5.0,0.0
626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,8.0,1.0,0.0,62.0,2.0,Mohammad Saleem,2.0,1.0,0.0


In [64]:
filtered_df.columns

Index(['BATTING', 'team_x', 'R_x', 'B', '4s_x', '6s_x', 'SR', '100s', '50s',
       'Match_id_x', 'Name', 'Born', 'Age', 'Batting_Style', 'Bowling_Style',
       'Playing_Role', 'National_Team', 'player_img', 'career_start',
       'career_latest', 'BOWLING', 'team_y', 'O', 'M', 'R_y', 'W', 'ECON',
       '0s', '4s_y', '6s_y', 'WD', 'NB', '5w_haul', 'balls_bowled',
       'Match_id_y', 'Player_Name', 'matches', 'catches', 'stumps'],
      dtype='object')

In [65]:
# rename columns

In [66]:
# RENAME : R_X, B, 4S_X, 6S_X, SR, o, m, R_Y, W, ECON, 0s, 4s_y, 6s_y, WD, NB
# drop: team_x, team_y, BATTING, BOWLING
# CHANGE VALUE: Playing_role, player_img

In [67]:
filtered_df = filtered_df.drop(columns=["team_x","team_y","BATTING","BOWLING","matches","Player_Name"],axis=0)

In [68]:
new_column_names = {'R_x':'Runs_scored_batting','B':"balls faced", '4s_x':"4s_batting", '6s_x':"6s_batting", 'SR':"strikerate",'Match_id_x':"Num_matches_batted", 'O':"Overs", 'M':"Maindens", 'R_y':"runs_given_baller", 'W':"wickets_baller", 'ECON':"economy_baller", '0s':"dot_balls_baller", '4s_y':"4s_baller", '6s_y':"6s_baller", 'WD':"wides_baller", 'NB':"NB_baller","Match_id_y":"Num_matches_bowled"}
filtered_df.rename(columns=new_column_names, inplace=True)

In [69]:
filtered_df.columns

Index(['Runs_scored_batting', 'balls faced', '4s_batting', '6s_batting',
       'strikerate', '100s', '50s', 'Num_matches_batted', 'Name', 'Born',
       'Age', 'Batting_Style', 'Bowling_Style', 'Playing_Role',
       'National_Team', 'player_img', 'career_start', 'career_latest', 'Overs',
       'Maindens', 'runs_given_baller', 'wickets_baller', 'economy_baller',
       'dot_balls_baller', '4s_baller', '6s_baller', 'wides_baller',
       'NB_baller', '5w_haul', 'balls_bowled', 'Num_matches_bowled', 'catches',
       'stumps'],
      dtype='object')

In [70]:
filtered_df["Playing_Role"].unique()

array(['Opening Batter', 'Top order Batter', 'Middle order Batter',
       'Wicketkeeper Batter', 'Bowling Allrounder', 'Bowler', 'Batter',
       'Allrounder', 'Batting Allrounder', nan], dtype=object)

In [71]:
filtered_df["batting_average"] = filtered_df["Runs_scored_batting"]/filtered_df["Num_matches_batted"]

In [72]:
def apply_conditions(row):
    if row['Playing_Role'] == 'Allrounder' or row['Playing_Role'] == 'Batting Allrounder' or row['Playing_Role'] == 'Bowling Allrounder' :
        if row['batting_average'] > 25:
            return 'Batting Allrounder'
        else:
            return 'Bowling Allrounder'
    else:
        return row['Playing_Role']
# Apply the custom function to create the new column
filtered_df['Playing_Role'] = filtered_df.apply(apply_conditions, axis=1)

In [73]:
filtered_df["Bowling_Style"].unique()

array(['Right arm Offbreak', 'Right arm Medium', 'Slow Left arm Orthodox',
       'Right arm Medium fast', 'Legbreak', 'Left arm Fast medium',
       'Right arm Fast', nan, 'Legbreak Googly', 'Left arm Fast',
       'Right arm Offbreak, Legbreak', 'Right arm Fast medium',
       'Left arm Wrist spin', 'Right arm Offbreak, Legbreak Googly',
       'Left arm Medium fast',
       'Right arm Offbreak, Slow Left arm Orthodox', 'Left arm Slow',
       'Right arm Medium, Right arm Offbreak', 'Left arm Medium',
       'Right arm Slow medium'], dtype=object)

In [74]:
filtered_df["Playing_Role"].unique()

array(['Opening Batter', 'Top order Batter', 'Middle order Batter',
       'Wicketkeeper Batter', 'Bowling Allrounder', 'Bowler', 'Batter',
       'Batting Allrounder', nan], dtype=object)

In [75]:
filtered_df[filtered_df["Playing_Role"]=="Batter"][["Name","National_Team"]]

,Name,National_Team
9,Babar Azam,Pakistan
107,Khaya Zondo,South Africa
174,James Vince,England
182,Wesley Barresi,Netherlands
231,Peter Handscomb,Australia
345,Mayank Agarwal,India
409,Suryakumar Yadav,India
432,Zubayr Hamza,South Africa
530,Ruturaj Gaikwad,India
550,Nuwanidu Fernando,Sri Lanka


In [76]:
def map_values(value):
    if value == 'Top order Batter':
        return 'Top order Batter'
    elif value == 'Opening Batter':
        return 'Top order Batter'
    elif value == 'Middle order Batter':
        return 'Middle order Batter'
    elif value == 'Wicketkeeper Batter':
        return 'Wicketkeeper Batter'
    elif value == 'Batter':
        return 'Batter'
    elif value == 'Batting Allrounder':
        return 'Batting Allrounder'
    elif value == 'Bowling Allrounder':
        return "Bowling Allrounder"
    else :
        return "Bowler"
    
# Apply the custom function to create the new column
filtered_df['Playing_Role_New'] = filtered_df['Playing_Role'].apply(map_values)

In [77]:
filtered_df["Playing_Role_New"].unique()

array(['Top order Batter', 'Middle order Batter', 'Wicketkeeper Batter',
       'Bowling Allrounder', 'Bowler', 'Batter', 'Batting Allrounder'],
      dtype=object)

In [78]:
filtered_df[filtered_df["Playing_Role"]=="Middle order Batter"][["Name","National_Team","batting_average","wickets_baller"]]

,Name,National_Team,batting_average,wickets_baller
2,Ross Taylor,New Zealand,47.710526,NaN
31,Steven Smith,Australia,43.513514,1.0
32,Travis Head,Australia,40.750000,2.0
41,Eoin Morgan,England,36.269231,NaN
58,Haris Sohail,Pakistan,43.681818,2.0
83,Sabbir Rahman,Bangladesh,19.333333,0.0
95,David Miller,South Africa,33.909091,NaN
113,Nasir Jamal,Afghanistan,18.666667,NaN
114,Najibullah Zadran,Afghanistan,27.851064,NaN
169,Asif Ali,Pakistan,23.875000,0.0


In [79]:
filtered_df[filtered_df["Playing_Role_New"]=="Batter"]

,Runs_scored_batting,balls faced,4s_batting,6s_batting,strikerate,100s,50s,Num_matches_batted,Name,Born,...,6s_baller,wides_baller,NB_baller,5w_haul,balls_bowled,Num_matches_bowled,catches,stumps,batting_average,Playing_Role_New
9,3315.0,3635.0,318.0,34.0,80.977258,11.0,19.0,62.0,Babar Azam,"October 15, 1994, Lahore, Punjab",...,NaN,NaN,NaN,NaN,NaN,NaN,45.0,0.0,53.467742,Batter
107,146.0,231.0,8.0,3.0,61.581667,0.0,1.0,6.0,Khaya Zondo,"March 07, 1990, Westville, Durban",...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,24.333333,Batter
174,512.0,580.0,64.0,3.0,79.221111,1.0,2.0,18.0,James Vince,"March 14, 1991, Cuckfield, Sussex",...,0.0,1.0,0.0,0.0,42.0,2.0,10.0,0.0,28.444444,Batter
182,410.0,479.0,44.0,6.0,67.642941,0.0,3.0,17.0,Wesley Barresi,"May 03, 1984, Johannesburg",...,0.0,0.0,0.0,0.0,12.0,2.0,20.0,8.0,24.117647,Batter
231,483.0,500.0,36.0,5.0,86.366154,1.0,3.0,13.0,Peter Handscomb,"April 26, 1991, Melbourne, Victoria",...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.0,37.153846,Batter
345,86.0,83.0,12.0,1.0,85.292000,0.0,0.0,5.0,Mayank Agarwal,"February 16, 1991, Bangalore, Karnataka",...,0.0,0.0,0.0,0.0,6.0,1.0,2.0,0.0,17.200000,Batter
409,399.0,399.0,43.0,5.0,90.985000,0.0,2.0,20.0,Suryakumar Yadav,"September 14, 1990, Mumbai, Maharashtra",...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.0,19.950000,Batter
432,56.0,79.0,5.0,1.0,70.880000,0.0,1.0,1.0,Zubayr Hamza,"June 19, 1995, Cape Town, Cape Province",...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,56.000000,Batter
530,19.0,42.0,1.0,0.0,45.230000,0.0,0.0,1.0,Ruturaj Gaikwad,"January 31, 1997, Pune, Maharashtra",...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,19.000000,Batter
550,75.0,105.0,10.0,0.0,59.652500,0.0,1.0,4.0,Nuwanidu Fernando,"October 13, 1999, Colombo",...,1.0,0.0,0.0,0.0,12.0,1.0,0.0,0.0,18.750000,Batter


In [80]:
updated_values = ["Top order Batter", "Middle order Batter","Top order Batter","Top order Batter","Middle order Batter","Top order Batter", 
                  "Middle order Batter","Middle order Batter","Top order Batter", "Top order Batter","Middle order Batter",
                  "Top order Batter","Middle order Batter"]

# Update the values in the DataFrame
filtered_df.loc[filtered_df["Playing_Role_New"] == "Batter", "Playing_Role_New"] = updated_values


In [81]:
filtered_df["Playing_Role_New"].unique()

array(['Top order Batter', 'Middle order Batter', 'Wicketkeeper Batter',
       'Bowling Allrounder', 'Bowler', 'Batting Allrounder'], dtype=object)

In [82]:
filtered_df.columns

Index(['Runs_scored_batting', 'balls faced', '4s_batting', '6s_batting',
       'strikerate', '100s', '50s', 'Num_matches_batted', 'Name', 'Born',
       'Age', 'Batting_Style', 'Bowling_Style', 'Playing_Role',
       'National_Team', 'player_img', 'career_start', 'career_latest', 'Overs',
       'Maindens', 'runs_given_baller', 'wickets_baller', 'economy_baller',
       'dot_balls_baller', '4s_baller', '6s_baller', 'wides_baller',
       'NB_baller', '5w_haul', 'balls_bowled', 'Num_matches_bowled', 'catches',
       'stumps', 'batting_average', 'Playing_Role_New'],
      dtype='object')

In [83]:
filtered_df["boundary_perc"] = ((filtered_df["4s_batting"]+filtered_df["6s_batting"])/(filtered_df["balls faced"]))*100.0

In [84]:
filtered_df["50_%"] = ((filtered_df["50s"])/(filtered_df["Num_matches_batted"]))*100
filtered_df["100_%"] = ((filtered_df["100s"])/(filtered_df["Num_matches_batted"]))*100

In [85]:
filtered_df["bowling_average"] = filtered_df.apply(lambda row: 2000 if row["wickets_baller"] < 1 else row['runs_given_baller'] / row['wickets_baller'], axis=1)


In [86]:
filtered_df["bowling_strikerate"] = filtered_df.apply(lambda row: 2000 if row["wickets_baller"] < 1 else row['balls_bowled'] / row['wickets_baller'], axis=1)


In [87]:
ranked_team_df = pd.DataFrame(columns=list(filtered_df.columns)+["Rank"])


In [88]:
ranked_team_df.columns

Index(['Runs_scored_batting', 'balls faced', '4s_batting', '6s_batting',
       'strikerate', '100s', '50s', 'Num_matches_batted', 'Name', 'Born',
       'Age', 'Batting_Style', 'Bowling_Style', 'Playing_Role',
       'National_Team', 'player_img', 'career_start', 'career_latest', 'Overs',
       'Maindens', 'runs_given_baller', 'wickets_baller', 'economy_baller',
       'dot_balls_baller', '4s_baller', '6s_baller', 'wides_baller',
       'NB_baller', '5w_haul', 'balls_bowled', 'Num_matches_bowled', 'catches',
       'stumps', 'batting_average', 'Playing_Role_New', 'boundary_perc',
       '50_%', '100_%', 'bowling_average', 'bowling_strikerate', 'Rank'],
      dtype='object')

#### Metrhod-1 Assigning weights (Intution)

In [89]:
# Define ranking metrics
ranking_metrics = {
    'Top order Batter': {'batting_average': 0.35, 'strikerate': 0.4,"boundary_perc":0.25}, 
    'Middle order Batter': {"boundary_perc": 0.2, 'batting_average': 0.45, 'strikerate': 0.35},
    'Wicketkeeper Batter': {'batting_average': 0.3, 'strikerate': 0.3, 'catches': 0.2,"stumps":0.2},
    'Batting Allrounder': {'strikerate': 0.25, 'batting_average': 0.3, 'boundary_perc': 0.25, 'wickets_baller': 0.3},
    'Bowling Allrounder': {'strikerate': 0.25, 'batting_average': 0.1, 'wickets_baller': 0.35, 'economy_baller': 0.3},
    'Bowler': {'wickets_baller': 0.6, 'economy_baller': 0.4}
}

# Loop over each team
for team in worldcup_teams:
    a = filtered_df[filtered_df["National_Team"] == team].copy()
    
    
    def calculate_rank(row, metrics):
        score = 0
        for metric, weight in metrics.items():
            if metric == 'economy_baller':
                score += (1 / row[metric]) * weight  # Lower economy is better
            else:
                score += row[metric] * weight
        return score

    a['Combined_Rank'] = a.apply(lambda row: calculate_rank(row, ranking_metrics[row['Playing_Role_New']]), axis=1)

    # Rank players from 1 to 10 within each playing role group
    a['Rank'] = a.groupby('Playing_Role_New')['Combined_Rank'].rank(method='min', ascending=False)

    # Sort the DataFrame first by 'Playing_Role_New' and then by 'Rank'
    a = a.sort_values(by=['Playing_Role_New', 'Rank'])

    print(team, len(a))

    ranked_team_df = pd.concat([ranked_team_df, a], ignore_index=True)

# Display the resulting DataFrame
print(ranked_team_df)

India 38
Australia 29
South Africa 31
Sri Lanka 42
Netherlands 28
New Zealand 34
Pakistan 32
Bangladesh 28
Afghanistan 28
England 37
     Runs_scored_batting  balls faced  4s_batting  6s_batting  strikerate  \
0                  982.0        907.0        83.0        30.0  100.276765   
1                  233.0        272.0        17.0         6.0   95.066667   
2                  135.0        220.0         8.0         0.0   63.251200   
3                   76.0        130.0         8.0         0.0   55.115833   
4                   88.0        106.0         7.0         3.0   67.332500   
..                   ...          ...         ...         ...         ...   
322               2617.0       2404.0       308.0        77.0   97.628070   
323                460.0        366.0        67.0         4.0  118.926667   
324                 61.0         76.0         3.0         1.0   80.260000   
325                454.0        505.0        46.0         6.0   79.200769   
326                1

In [90]:
ranked_team_df

,Runs_scored_batting,balls faced,4s_batting,6s_batting,strikerate,100s,50s,Num_matches_batted,Name,Born,...,stumps,batting_average,Playing_Role_New,boundary_perc,50_%,100_%,bowling_average,bowling_strikerate,Rank,Combined_Rank
0,982.0,907.0,83.0,30.0,100.276765,0.0,5.0,34.0,Hardik Pandya,"October 11, 1993, Choryasi, Gujarat",...,0.0,28.882353,Batting Allrounder,12.458655,14.705882,0.000000,39.702703,42.054054,1.0,47.948561
1,233.0,272.0,17.0,6.0,95.066667,0.0,1.0,9.0,Washington Sundar,"October 05, 1999, Chennai, Tamil Nadu",...,0.0,25.888889,Batting Allrounder,8.455882,11.111111,0.000000,24.666667,30.400000,2.0,38.147304
2,135.0,220.0,8.0,0.0,63.251200,0.0,0.0,25.0,Kuldeep Yadav,"December 14, 1994, Kanpur, Uttar Pradesh",...,0.0,5.400000,Bowler,3.636364,0.000000,0.000000,28.243243,32.000000,1.0,66.675373
3,76.0,130.0,8.0,0.0,55.115833,0.0,0.0,12.0,Yuzvendra Chahal,"July 23, 1990, Jind, Haryana",...,0.0,6.333333,Bowler,6.153846,0.000000,0.000000,27.723404,30.521277,2.0,56.471736
4,88.0,106.0,7.0,3.0,67.332500,0.0,0.0,20.0,Mohammed Shami,"September 03, 1990, Amroha, Uttar Pradesh",...,0.0,4.400000,Bowler,9.433962,0.000000,0.000000,26.704225,27.647887,3.0,42.669599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,2617.0,2404.0,308.0,77.0,97.628070,9.0,10.0,57.0,Jonny Bairstow,"September 26, 1989, Bradford, Yorkshire",...,3.0,45.912281,Wicketkeeper Batter,16.014975,17.543860,15.789474,NaN,NaN,2.0,53.262105
323,460.0,366.0,67.0,4.0,118.926667,1.0,2.0,12.0,Phil Salt,"August 28, 1996, Bodelwyddan, North Wales",...,0.0,38.333333,Wicketkeeper Batter,19.398907,16.666667,8.333333,NaN,NaN,3.0,47.978000
324,61.0,76.0,3.0,1.0,80.260000,0.0,1.0,1.0,Ben Foakes,"February 15, 1993, Colchester, Essex",...,1.0,61.000000,Wicketkeeper Batter,5.263158,100.000000,0.000000,NaN,NaN,4.0,42.978000
325,454.0,505.0,46.0,6.0,79.200769,1.0,3.0,13.0,Sam Billings,"June 15, 1991, Pembury, Kent",...,1.0,34.923077,Wicketkeeper Batter,10.297030,23.076923,7.692308,NaN,NaN,5.0,38.237154


In [91]:
df_players['Is_Captain'] = 'No'
captains_list = ["Rohit Sharma","Scott Edwards","Temba Bavuma","Jos Butler","Babar Azam",
                 "Dasun Shanaka","Hashmatullah Shahidi","Pat Cummins","Tamim Iqbal", "Kane Williamson"]
# Iterate through the list of captains and mark the corresponding rows as 'Yes'
for captain_name in captains_list:
    ranked_team_df.loc[ranked_team_df['Name'] == captain_name, 'Is_Captain'] = 'Yes'

# Print the updated DataFrame
ranked_team_df.columns

Index(['Runs_scored_batting', 'balls faced', '4s_batting', '6s_batting',
       'strikerate', '100s', '50s', 'Num_matches_batted', 'Name', 'Born',
       'Age', 'Batting_Style', 'Bowling_Style', 'Playing_Role',
       'National_Team', 'player_img', 'career_start', 'career_latest', 'Overs',
       'Maindens', 'runs_given_baller', 'wickets_baller', 'economy_baller',
       'dot_balls_baller', '4s_baller', '6s_baller', 'wides_baller',
       'NB_baller', '5w_haul', 'balls_bowled', 'Num_matches_bowled', 'catches',
       'stumps', 'batting_average', 'Playing_Role_New', 'boundary_perc',
       '50_%', '100_%', 'bowling_average', 'bowling_strikerate', 'Rank',
       'Combined_Rank', 'Is_Captain'],
      dtype='object')

In [92]:
a = ranked_team_df[ranked_team_df["National_Team"] == "Australia"]
a

,Runs_scored_batting,balls faced,4s_batting,6s_batting,strikerate,100s,50s,Num_matches_batted,Name,Born,...,batting_average,Playing_Role_New,boundary_perc,50_%,100_%,bowling_average,bowling_strikerate,Rank,Combined_Rank,Is_Captain
38,1455.0,1154.0,129.0,59.0,133.027174,1.0,8.0,46.0,Glenn Maxwell,"October 14, 1988, Kew, Melbourne, Victoria",...,31.630435,Batting Allrounder,16.291161,17.391304,2.173913,84.800000,90.666667,1.0,51.318714,NaN
39,766.0,857.0,67.0,24.0,88.015417,0.0,6.0,24.0,Mitchell Marsh,"October 20, 1991, Attadale, Perth",...,31.916667,Batting Allrounder,10.618436,25.000000,0.000000,33.076923,39.923077,2.0,38.133463,NaN
40,302.0,340.0,22.0,8.0,89.078333,0.0,1.0,12.0,Cameron Green,"June 03, 1999",...,25.166667,Batting Allrounder,8.823529,8.333333,0.000000,34.272727,40.909091,3.0,35.325466,NaN
41,847.0,1018.0,52.0,2.0,76.267143,1.0,6.0,28.0,Marnus Labuschagne,"June 22, 1994, Klerksdorp, North West Province...",...,30.250000,Batting Allrounder,5.304519,21.428571,3.571429,98.500000,92.500000,4.0,30.067915,NaN
42,160.0,253.0,15.0,0.0,55.830417,0.0,0.0,24.0,Adam Zampa,"March 31, 1992, Shellharbour, New South Wales",...,6.666667,Bowler,5.928854,0.000000,0.000000,27.307692,30.747253,1.0,54.673743,NaN
43,224.0,280.0,15.0,6.0,75.106333,0.0,0.0,30.0,Mitchell Starc,"January 30, 1990, Baulkham Hills, Sydney, New ...",...,7.466667,Bowler,7.500000,0.000000,0.000000,25.188235,27.258824,2.0,51.072699,NaN
44,196.0,259.0,12.0,2.0,61.433333,0.0,0.0,30.0,Pat Cummins,"May 08, 1993, Westmead, Sydney",...,6.533333,Bowler,5.405405,0.000000,0.000000,26.223881,31.194030,3.0,40.280276,Yes
45,60.0,71.0,4.0,1.0,52.479286,0.0,0.0,14.0,Josh Hazlewood,"January 08, 1991, Tamworth, New South Wales",...,4.285714,Bowler,7.042254,0.000000,0.000000,28.159091,38.045455,4.0,26.492510,NaN
46,93.0,107.0,7.0,0.0,66.700000,0.0,0.0,9.0,Jhye Richardson,"September 20, 1996, Murdoch, Western Australia",...,10.333333,Bowler,6.542056,0.000000,0.000000,29.370370,30.000000,5.0,16.267904,NaN
47,45.0,42.0,5.0,2.0,95.097143,0.0,0.0,7.0,Kane Richardson,"February 12, 1991, Eudunda, South Australia",...,6.428571,Bowler,16.666667,0.000000,0.000000,30.111111,29.611111,6.0,10.865746,NaN


### Method - 2 Claculating weights (Logistic Regression)

##### Instead of assigning weights through intution I also tried using logistic regression to find best set of coefficients for each group thr analysis is shown below

##### For Top order batsmen (Removing bowlers)

In [93]:
filtered_df.columns

Index(['Runs_scored_batting', 'balls faced', '4s_batting', '6s_batting',
       'strikerate', '100s', '50s', 'Num_matches_batted', 'Name', 'Born',
       'Age', 'Batting_Style', 'Bowling_Style', 'Playing_Role',
       'National_Team', 'player_img', 'career_start', 'career_latest', 'Overs',
       'Maindens', 'runs_given_baller', 'wickets_baller', 'economy_baller',
       'dot_balls_baller', '4s_baller', '6s_baller', 'wides_baller',
       'NB_baller', '5w_haul', 'balls_bowled', 'Num_matches_bowled', 'catches',
       'stumps', 'batting_average', 'Playing_Role_New', 'boundary_perc',
       '50_%', '100_%', 'bowling_average', 'bowling_strikerate'],
      dtype='object')

In [94]:
check2 = filtered_df[["Name", "Playing_Role_New","strikerate",
       'boundary_perc', '50_%','100_%',"batting_average",'Runs_scored_batting']].copy()

In [95]:
# Drop rows with "Bowler" in the "Playing_Role_New" column
check = check2[check2['Playing_Role_New'] != 'Bowler']

In [96]:
check.head()

,Name,Playing_Role_New,strikerate,boundary_perc,50_%,100_%,batting_average,Runs_scored_batting
0,Martin Guptill,Top order Batter,80.434082,10.795181,14.285714,12.244898,36.591837,1793.0
1,Kane Williamson,Top order Batter,68.361951,6.507680,24.390244,9.756098,44.829268,1838.0
2,Ross Taylor,Middle order Batter,78.242632,8.447266,31.578947,10.526316,47.710526,1813.0
3,Tom Latham,Wicketkeeper Batter,70.362807,8.028846,21.052632,5.263158,32.596491,1858.0
4,Henry Nicholls,Top order Batter,66.462083,8.055724,20.833333,2.083333,27.562500,1323.0


In [97]:
check.reset_index(drop=True)
check.fillna(0, inplace = True)
check

C:\Users\ishan\AppData\Local\Temp\ipykernel_35168\1417566238.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check.fillna(0, inplace = True)


,Name,Playing_Role_New,strikerate,boundary_perc,50_%,100_%,batting_average,Runs_scored_batting
0,Martin Guptill,Top order Batter,80.434082,10.795181,14.285714,12.244898,36.591837,1793.0
1,Kane Williamson,Top order Batter,68.361951,6.507680,24.390244,9.756098,44.829268,1838.0
2,Ross Taylor,Middle order Batter,78.242632,8.447266,31.578947,10.526316,47.710526,1813.0
3,Tom Latham,Wicketkeeper Batter,70.362807,8.028846,21.052632,5.263158,32.596491,1858.0
4,Henry Nicholls,Top order Batter,66.462083,8.055724,20.833333,2.083333,27.562500,1323.0
...,...,...,...,...,...,...,...,...
607,Cole McConchie,Batting Allrounder,101.120000,13.095238,33.333333,0.000000,32.666667,98.0
609,Rony Talukdar,Top order Batter,28.570000,7.142857,0.000000,0.000000,4.000000,4.0
611,Dushan Hemantha,Bowling Allrounder,110.000000,10.000000,0.000000,0.000000,22.000000,22.0
627,Sahan Arachchige,Top order Batter,80.280000,5.633803,100.000000,0.000000,57.000000,57.0


In [98]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Preprocess the data
le = LabelEncoder()
check['Playing_Role_New'] = check['Playing_Role_New'].apply(lambda x: 'Top order' if x == 'Top order Batter' else 'Other')
check['Playing_Role_New'] = le.fit_transform(check['Playing_Role_New'])

X = check.drop(['Name', 'Playing_Role_New'], axis=1)
y = check['Playing_Role_New']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

C:\Users\ishan\AppData\Local\Temp\ipykernel_35168\1701514466.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['Playing_Role_New'] = check['Playing_Role_New'].apply(lambda x: 'Top order' if x == 'Top order Batter' else 'Other')
C:\Users\ishan\AppData\Local\Temp\ipykernel_35168\1701514466.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['Playing_Role_New'] = le.fit_transform(check['Playing_Role_New'])


LogisticRegression()

In [99]:
y_pred = model.predict(X_test)

In [100]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=le.classes_)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy: 0.7560975609756098
Classification Report:
               precision    recall  f1-score   support

       Other       0.78      0.94      0.85        31
   Top order       0.50      0.20      0.29        10

    accuracy                           0.76        41
   macro avg       0.64      0.57      0.57        41
weighted avg       0.71      0.76      0.71        41



In [101]:
# Get coefficients and feature names
coefficients = model.coef_[0]
feature_names = X.columns

# Print coefficients and their corresponding feature names
print("Model Coefficients:")
for feature, coef in zip(feature_names, coefficients):
    print(feature, ':', coef)

Model Coefficients:
strikerate : -1.597671127959683
boundary_perc : 0.6069683066351771
50_% : -0.03633892136701006
100_% : -0.011224817606443182
batting_average : 0.7256981409788312
Runs_scored_batting : 0.3467972460316084


In [102]:
import numpy as np

In [103]:
#Step 1: Normalize coefficients between 0 and 1
top_order = np.abs(coefficients) / np.sum(np.abs(coefficients))

print("Original Coefficients:", coefficients)
print("Normalized Coefficients:", top_order)

Original Coefficients: [-1.59767113  0.60696831 -0.03633892 -0.01122482  0.72569814  0.34679725]
Normalized Coefficients: [0.48054616 0.18256341 0.01092999 0.00337619 0.21827487 0.10430938]


### Middle Order Batsmen

In [104]:
check2 = filtered_df[["Name", "Playing_Role_New","strikerate",
       'boundary_perc', '50_%','100_%',"batting_average",'Runs_scored_batting']].copy()

In [105]:
check = check2[check2['Playing_Role_New'] != 'Bowler']
check.reset_index(drop=True)
check.fillna(0, inplace = True)

C:\Users\ishan\AppData\Local\Temp\ipykernel_35168\2103000716.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check.fillna(0, inplace = True)


In [106]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Preprocess the data
le = LabelEncoder()
check['Playing_Role_New'] = check['Playing_Role_New'].apply(lambda x: 'Middle order' if x == 'Middle order Batter' else 'Other')
check['Playing_Role_New'] = le.fit_transform(check['Playing_Role_New'])

X = check.drop(['Name', 'Playing_Role_New'], axis=1)
y = check['Playing_Role_New']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

C:\Users\ishan\AppData\Local\Temp\ipykernel_35168\3292371394.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['Playing_Role_New'] = check['Playing_Role_New'].apply(lambda x: 'Middle order' if x == 'Middle order Batter' else 'Other')
C:\Users\ishan\AppData\Local\Temp\ipykernel_35168\3292371394.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['Playing_Role_New'] = le.fit_transform(check['Playing_Role_New'])


LogisticRegression()

In [107]:
y_pred = model.predict(X_test)

In [108]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=le.classes_)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy: 0.9024390243902439
Classification Report:
               precision    recall  f1-score   support

Middle order       0.00      0.00      0.00         4
       Other       0.90      1.00      0.95        37

    accuracy                           0.90        41
   macro avg       0.45      0.50      0.47        41
weighted avg       0.81      0.90      0.86        41



C:\Users\ishan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ishan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ishan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
# Get coefficients and feature names
coefficients = model.coef_[0]
feature_names = X.columns

# Print coefficients and their corresponding feature names
print("Model Coefficients:")
for feature, coef in zip(feature_names, coefficients):
    print(feature, ':', coef)

Model Coefficients:
strikerate : -0.22732248577088232
boundary_perc : -0.16289575009318571
50_% : -0.6140734581777161
100_% : -0.15200092188066702
batting_average : 0.5629491224408395
Runs_scored_batting : 0.3299794920628158


In [110]:
#Step 1: Normalize coefficients between 0 and 1
middle_order = np.abs(coefficients) / np.sum(np.abs(coefficients))

print("Original Coefficients:", coefficients)
print("Normalized Coefficients:", middle_order)

Original Coefficients: [-0.22732249 -0.16289575 -0.61407346 -0.15200092  0.56294912  0.32997949]
Normalized Coefficients: [0.11093116 0.07949154 0.29966187 0.07417497 0.27471369 0.16102678]


In [111]:
filtered_df.head()

,Runs_scored_batting,balls faced,4s_batting,6s_batting,strikerate,100s,50s,Num_matches_batted,Name,Born,...,Num_matches_bowled,catches,stumps,batting_average,Playing_Role_New,boundary_perc,50_%,100_%,bowling_average,bowling_strikerate
0,1793.0,2075.0,175.0,49.0,80.434082,6.0,7.0,49.0,Martin Guptill,"September 30, 1986, Auckland",...,NaN,104.0,0.0,36.591837,Top order Batter,10.795181,14.285714,12.244898,NaN,NaN
1,1838.0,2474.0,149.0,12.0,68.361951,4.0,10.0,41.0,Kane Williamson,"August 08, 1990, Tauranga",...,6.0,64.0,0.0,44.829268,Top order Batter,6.507680,24.390244,9.756098,33.75,49.5
2,1813.0,2048.0,147.0,26.0,78.242632,4.0,12.0,38.0,Ross Taylor,"March 08, 1984, Lower Hutt, Wellington",...,NaN,142.0,0.0,47.710526,Middle order Batter,8.447266,31.578947,10.526316,NaN,NaN
3,1858.0,2080.0,142.0,25.0,70.362807,3.0,12.0,57.0,Tom Latham,"April 02, 1992, Christchurch, Canterbury",...,NaN,111.0,15.0,32.596491,Wicketkeeper Batter,8.028846,21.052632,5.263158,NaN,NaN
4,1323.0,1651.0,121.0,12.0,66.462083,1.0,10.0,48.0,Henry Nicholls,"November 15, 1991, Christchurch",...,NaN,24.0,0.0,27.562500,Top order Batter,8.055724,20.833333,2.083333,NaN,NaN


### Wicket Keeper Batter

In [112]:
check2 = filtered_df[["Name", "Playing_Role_New","strikerate",
       'boundary_perc', '50_%','100_%',"batting_average",'Runs_scored_batting']].copy()

check = check2[check2['Playing_Role_New'] != 'Bowler']
check.reset_index(drop=True)
check.fillna(0, inplace = True)

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Preprocess the data
le = LabelEncoder()
check['Playing_Role_New'] = check['Playing_Role_New'].apply(lambda x: 'Middle order' if x == 'Wicketkeeper Batter' else 'Other')
check['Playing_Role_New'] = le.fit_transform(check['Playing_Role_New'])

X = check.drop(['Name', 'Playing_Role_New'], axis=1)
y = check['Playing_Role_New']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=le.classes_)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

# Get coefficients and feature names
coefficients = model.coef_[0]
feature_names = X.columns

# Print coefficients and their corresponding feature names
print("Model Coefficients:")
for feature, coef in zip(feature_names, coefficients):
    print(feature, ':', coef)

#Step 1: Normalize coefficients between 0 and 1
wicket_keeper_order = np.abs(coefficients) / np.sum(np.abs(coefficients))

print("Original Coefficients:", coefficients)
print("Normalized Coefficients:", wicket_keeper_order)

Accuracy: 0.8048780487804879
Classification Report:
               precision    recall  f1-score   support

Middle order       0.00      0.00      0.00         8
       Other       0.80      1.00      0.89        33

    accuracy                           0.80        41
   macro avg       0.40      0.50      0.45        41
weighted avg       0.65      0.80      0.72        41

Model Coefficients:
strikerate : -0.04154880885578056
boundary_perc : -0.044265828118887
50_% : 0.574573222703579
100_% : 0.1290926164310936
batting_average : -0.9442973884413324
Runs_scored_batting : 0.17518901986233376
Original Coefficients: [-0.04154881 -0.04426583  0.57457322  0.12909262 -0.94429739  0.17518902]
Normalized Coefficients: [0.02176508 0.02318837 0.30098648 0.06762434 0.4946641  0.09177164]


C:\Users\ishan\AppData\Local\Temp\ipykernel_35168\1372791605.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check.fillna(0, inplace = True)
C:\Users\ishan\AppData\Local\Temp\ipykernel_35168\1372791605.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['Playing_Role_New'] = check['Playing_Role_New'].apply(lambda x: 'Middle order' if x == 'Wicketkeeper Batter' else 'Other')
C:\Users\ishan\AppData\Local\Temp\ipykernel_35168\1372791605.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [113]:
filtered_df.columns

Index(['Runs_scored_batting', 'balls faced', '4s_batting', '6s_batting',
       'strikerate', '100s', '50s', 'Num_matches_batted', 'Name', 'Born',
       'Age', 'Batting_Style', 'Bowling_Style', 'Playing_Role',
       'National_Team', 'player_img', 'career_start', 'career_latest', 'Overs',
       'Maindens', 'runs_given_baller', 'wickets_baller', 'economy_baller',
       'dot_balls_baller', '4s_baller', '6s_baller', 'wides_baller',
       'NB_baller', '5w_haul', 'balls_bowled', 'Num_matches_bowled', 'catches',
       'stumps', 'batting_average', 'Playing_Role_New', 'boundary_perc',
       '50_%', '100_%', 'bowling_average', 'bowling_strikerate'],
      dtype='object')

In [114]:
#bowling_metrics - bowling_average, bowling_strikerate, economy_baller, wickets_baller

### Creating Final Dataset

In [190]:
ranked_team_df = pd.DataFrame(columns=list(filtered_df.columns)+["Rank"])

In [191]:
# Define ranking metrics

ranking_metrics = {
    'Top order Batter': {'batting_average': top_order[4], 'strikerate': top_order[0],"boundary_perc":top_order[1],
                         "50_%":top_order[2], "100_%":top_order[3],"Runs_scored_batting":top_order[5]}, 
    'Middle order Batter': {'batting_average': middle_order[4], 'strikerate': middle_order[0],"boundary_perc":middle_order[1], 
                            "50_%":middle_order[2], "100_%":middle_order[3], "Runs_scored_batting":middle_order[5]},
    'Wicketkeeper Batter': {'batting_average': wicket_keeper_order[4], 'strikerate': wicket_keeper_order[0],
                            "boundary_perc":wicket_keeper_order[1], "50_%":wicket_keeper_order[2], 
                            "100_%":wicket_keeper_order[3],"Runs_scored_batting":wicket_keeper_order[5]},
    
    'Batting Allrounder': {'batting_average': 0.12, 'strikerate': 0.24,"boundary_perc":0.12,"Num_matches_batted":0.05,
                           "50_%":0, "100_%":0, "Runs_scored_batting":0.06,'wickets_baller': 0.04, 
                           'economy_baller': 0.12,"bowling_average":0.12,"bowling_strikerate":0.12,'Num_matches_bowled':.05},
    
    'Bowling Allrounder': {'batting_average':0.08 , 'strikerate': 0.16,"boundary_perc":0.08, "Num_matches_batted":0.05,
                           "50_%":0, "100_%":0, "Runs_scored_batting":0.04,'wickets_baller': 0.05, 
                           'economy_baller': 0.18,"bowling_average":0.18,"bowling_strikerate":0.18,'Num_matches_bowled':.05},
    
    'Bowler': {'economy_baller': 0.3,"bowling_average":0.44,"bowling_strikerate":0.25, "wickets_baller":0.01}
}

# Loop over each team
for team in worldcup_teams:
    a = filtered_df[filtered_df["National_Team"] == team].copy()
    
    
    def calculate_rank(row, metrics):
        score = 0
        for metric, weight in metrics.items():
            if metric == 'economy_baller' or metric == 'bowling_average' or metric == "bowling_strikerate" :
                score += (1 / row[metric]) * weight  # Lower economy is better
            else:
                score += row[metric] * weight
        return score

    a['Combined_Rank'] = a.apply(lambda row: calculate_rank(row, ranking_metrics[row['Playing_Role_New']]), axis=1)

    # Rank players from 1 to 10 within each playing role group
    a['Rank'] = a.groupby('Playing_Role_New')['Combined_Rank'].rank(method='min', ascending=False)

    # Sort the DataFrame first by 'Playing_Role_New' and then by 'Rank'
    a = a.sort_values(by=['Playing_Role_New', 'Rank'])

    #print(team, len(a))

    ranked_team_df = pd.concat([ranked_team_df, a], ignore_index=True)

# Display the resulting DataFrame
#print(ranked_team_df)

### Example of How data will look like

In [193]:
ranked_team_df[ranked_team_df["National_Team"]=="Australia"]

,Runs_scored_batting,balls faced,4s_batting,6s_batting,strikerate,100s,50s,Num_matches_batted,Name,Born,...,stumps,batting_average,Playing_Role_New,boundary_perc,50_%,100_%,bowling_average,bowling_strikerate,Rank,Combined_Rank
38,1455.0,1154.0,129.0,59.0,133.027174,1.0,8.0,46.0,Glenn Maxwell,"October 14, 1988, Kew, Melbourne, Victoria",...,0.0,31.630435,Batting Allrounder,16.291161,17.391304,2.173913,84.800000,90.666667,1.0,129.900058
39,847.0,1018.0,52.0,2.0,76.267143,1.0,6.0,28.0,Marnus Labuschagne,"June 22, 1994, Klerksdorp, North West Province...",...,0.0,30.250000,Batting Allrounder,5.304519,21.428571,3.571429,98.500000,92.500000,2.0,75.339590
40,766.0,857.0,67.0,24.0,88.015417,0.0,6.0,24.0,Mitchell Marsh,"October 20, 1991, Attadale, Perth",...,0.0,31.916667,Batting Allrounder,10.618436,25.000000,0.000000,33.076923,39.923077,3.0,74.887273
41,302.0,340.0,22.0,8.0,89.078333,0.0,1.0,12.0,Cameron Green,"June 03, 1999",...,0.0,25.166667,Batting Allrounder,8.823529,8.333333,0.000000,34.272727,40.909091,4.0,45.397242
42,160.0,253.0,15.0,0.0,55.830417,0.0,0.0,24.0,Adam Zampa,"March 31, 1992, Shellharbour, New South Wales",...,0.0,6.666667,Bowler,5.928854,0.000000,0.000000,27.307692,30.747253,1.0,0.989551
43,224.0,280.0,15.0,6.0,75.106333,0.0,0.0,30.0,Mitchell Starc,"January 30, 1990, Baulkham Hills, Sydney, New ...",...,0.0,7.466667,Bowler,7.500000,0.000000,0.000000,25.188235,27.258824,2.0,0.931164
44,196.0,259.0,12.0,2.0,61.433333,0.0,0.0,30.0,Pat Cummins,"May 08, 1993, Westmead, Sydney",...,0.0,6.533333,Bowler,5.405405,0.000000,0.000000,26.223881,31.194030,3.0,0.755000
45,60.0,71.0,4.0,1.0,52.479286,0.0,0.0,14.0,Josh Hazlewood,"January 08, 1991, Tamworth, New South Wales",...,0.0,4.285714,Bowler,7.042254,0.000000,0.000000,28.159091,38.045455,4.0,0.531579
46,93.0,107.0,7.0,0.0,66.700000,0.0,0.0,9.0,Jhye Richardson,"September 20, 1996, Murdoch, Western Australia",...,0.0,10.333333,Bowler,6.542056,0.000000,0.000000,29.370370,30.000000,5.0,0.344242
47,45.0,42.0,5.0,2.0,95.097143,0.0,0.0,7.0,Kane Richardson,"February 12, 1991, Eudunda, South Australia",...,0.0,6.428571,Bowler,16.666667,0.000000,0.000000,30.111111,29.611111,6.0,0.252365


In [194]:
ranked_team_df.to_csv("ranked_best_players.csv")

In [199]:
ranked_team_df.columns

Index(['Runs_scored_batting', 'balls faced', '4s_batting', '6s_batting',
       'strikerate', '100s', '50s', 'Num_matches_batted', 'Name', 'Born',
       'Age', 'Batting_Style', 'Bowling_Style', 'Playing_Role',
       'National_Team', 'player_img', 'career_start', 'career_latest', 'Overs',
       'Maindens', 'runs_given_baller', 'wickets_baller', 'economy_baller',
       'dot_balls_baller', '4s_baller', '6s_baller', 'wides_baller',
       'NB_baller', '5w_haul', 'balls_bowled', 'Num_matches_bowled', 'catches',
       'stumps', 'batting_average', 'Playing_Role_New', 'boundary_perc',
       '50_%', '100_%', 'bowling_average', 'bowling_strikerate', 'Rank',
       'Combined_Rank'],
      dtype='object')

### Inserting Missing images of Players

In [207]:
df_player_links = pd.read_csv("player_links_data.csv")

In [208]:
df_player_links.head()

,Player_name,Player_link,odi_year
0,Martin Guptill,/cricketers/martin-guptill-226492,2018-2018
1,Colin Munro,/cricketers/colin-munro-232359,2018-2018
2,Kane Williamson,/cricketers/kane-williamson-277906,2018-2018
3,Ross Taylor,/cricketers/ross-taylor-38699,2018-2018
4,Tom Latham,/cricketers/tom-latham-388802,2018-2018


In [209]:
df_player_links = df_player_links.drop(columns = ["odi_year"])

In [218]:
df_player_links = df_player_links.drop_duplicates()

In [219]:
ranked_team_df["player_img"].isna().sum()

193

In [220]:
len(ranked_team_df)

327

In [221]:
merged_final = pd.merge(ranked_team_df, df_player_links, left_on=["Name"], right_on=["Player_name"], how='inner')
print(len(merged_final))

329


In [222]:
merged_final

,Runs_scored_batting,balls faced,4s_batting,6s_batting,strikerate,100s,50s,Num_matches_batted,Name,Born,...,Playing_Role_New,boundary_perc,50_%,100_%,bowling_average,bowling_strikerate,Rank,Combined_Rank,Player_name,Player_link
0,982.0,907.0,83.0,30.0,100.276765,0.0,5.0,34.0,Hardik Pandya,"October 11, 1993, Choryasi, Gujarat",...,Batting Allrounder,12.458655,14.705882,0.000000,39.702703,42.054054,1.0,93.053225,Hardik Pandya,/cricketers/hardik-pandya-625371
1,233.0,272.0,17.0,6.0,95.066667,0.0,1.0,9.0,Washington Sundar,"October 05, 1999, Chennai, Tamil Nadu",...,Batting Allrounder,8.455882,11.111111,0.000000,24.666667,30.400000,2.0,42.652250,Washington Sundar,/cricketers/washington-sundar-719715
2,135.0,220.0,8.0,0.0,63.251200,0.0,0.0,25.0,Kuldeep Yadav,"December 14, 1994, Kanpur, Uttar Pradesh",...,Bowler,3.636364,0.000000,0.000000,28.243243,32.000000,1.0,1.189921,Kuldeep Yadav,/cricketers/kuldeep-yadav-559235
3,76.0,130.0,8.0,0.0,55.115833,0.0,0.0,12.0,Yuzvendra Chahal,"July 23, 1990, Jind, Haryana",...,Bowler,6.153846,0.000000,0.000000,27.723404,30.521277,2.0,1.017864,Yuzvendra Chahal,/cricketers/yuzvendra-chahal-430246
4,88.0,106.0,7.0,3.0,67.332500,0.0,0.0,20.0,Mohammed Shami,"September 03, 1990, Amroha, Uttar Pradesh",...,Bowler,9.433962,0.000000,0.000000,26.704225,27.647887,3.0,0.787719,Mohammed Shami,/cricketers/mohammed-shami-481896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,2142.0,1815.0,162.0,84.0,102.062727,7.0,10.0,55.0,Jos Buttler,"September 08, 1990, Taunton, Somerset",...,Wicketkeeper Batter,13.553719,18.181818,12.727273,NaN,NaN,2.0,224.708616,Jos Buttler,/cricketers/jos-buttler-308967
325,460.0,366.0,67.0,4.0,118.926667,1.0,2.0,12.0,Phil Salt,"August 28, 1996, Bodelwyddan, North Wales",...,Wicketkeeper Batter,19.398907,16.666667,8.333333,NaN,NaN,3.0,69.795332,Phil Salt,/cricketers/phil-salt-669365
326,454.0,505.0,46.0,6.0,79.200769,1.0,3.0,13.0,Sam Billings,"June 15, 1991, Pembury, Kent",...,Wicketkeeper Batter,10.297030,23.076923,7.692308,NaN,NaN,4.0,68.368128,Sam Billings,/cricketers/sam-billings-297628
327,61.0,76.0,3.0,1.0,80.260000,0.0,1.0,1.0,Ben Foakes,"February 15, 1993, Colchester, Essex",...,Wicketkeeper Batter,5.263158,100.000000,0.000000,NaN,NaN,5.0,67.740137,Ben Foakes,/cricketers/ben-foakes-364788


In [281]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--no-sandbox")  # Optional: Disable sandbox mode
chrome_options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Specify your Chrome binary location



In [282]:
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

for index, row in merged_final.iterrows():
    
    
    url = "https://www.espncricinfo.com" + row["Player_link"]
    
    # Open the webpage
    driver.get(url)

    # Find the <img> tag with a specific alt attribute value
    alt_value = row["Name"]  # Replace with the alt value you are looking for
    
    t=20

    while True:
        try:
            # Wait for the image to be visible and loaded
            img_tag = WebDriverWait(driver, t).until(
                EC.visibility_of_element_located((By.XPATH, f"//img[@alt='{alt_value}']"))
            )

            # Get the src attribute of the <img> tag
            img_src = img_tag.get_attribute("src")

            if "lazyimage" in img_src:
                if t>60:
                    break
                t+=5
                continue 
                
            else:
                print("Image Source:", img_src)
                merged_final.at[index,"player_img"] = img_src
                break   
        except Exception as e:
            print("Image not found or failed to load.")
            print(e)
            break


# Close the browser
driver.quit()

Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_640,q_50/lsci/db/PICTURES/CMS/319900/319938.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_640,q_50/lsci/db/PICTURES/CMS/322600/322619.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_640,q_50/lsci/db/PICTURES/CMS/319900/319943.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_640,q_50/lsci/db/PICTURES/CMS/319900/319955.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/316600/316691.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/319900/319940.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/316600/316658.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322600/322696.png
Image Source: https://img1.hscicdn.com/image/upload/

Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/324200/324217.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322400/322461.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/348500/348579.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/316600/316608.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322000/322096.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322000/322085.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/329700/329749.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/320300/320334.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w

Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/331100/331170.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/324200/324241.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_h_100_2x/lsci/db/PICTURES/CMS/171300/171335.1.jpg
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_h_100_2x/lsci/db/PICTURES/CMS/362700/362791.1.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322600/322644.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_h_100_2x/lsci/db/PICTURES/CMS/362800/362803.1.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_h_100_2x/lsci/db/PICTURES/CMS/362700/362798.1.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/325900/325907.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURE

Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322100/322154.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/316600/316650.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322100/322192.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322100/322153.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322100/322168.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_h_100_2x/lsci/db/PICTURES/CMS/313600/313620.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322200/322204.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/320400/320448.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/ls

Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/316600/316660.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322900/322929.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/322900/322910.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/323000/323019.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/325900/325908.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_h_100_2x/lsci/db/PICTURES/CMS/349500/349594.4.jpg
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/323100/323105.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/lsci/db/PICTURES/CMS/324300/324359.png
Image Source: https://img1.hscicdn.com/image/upload/f_auto,t_ds_square_w_320/

In [283]:
merged_final["player_img"].isna().sum()

9

In [285]:
merged_final.to_csv("ranked_best_players.csv")